In [1]:
##Importar las librerías requeridas para el análisis
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
##Definir tamaño máximo de filas a mostrar
pd.set_option('display.max_rows', 100)

In [3]:
##Importar el dataset del CSV
automotores_df = pd.read_csv('../data/hurto_motocicletasV2.csv')

In [4]:
#Visualizamos un parte del dataset
automotores_df.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,TIPO DE HURTO,CANTIDAD
0,ANTIOQUIA,MEDELLÍN,5001,ARMA DE FUEGO,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
1,ANTIOQUIA,OPACABANA,5212,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
2,ANTIOQUIA,MEDELLÍN,5001,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
3,CUNDINAMARCA,BOGOTÁ D.C.,11001,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
4,VALLE,ALI,76001,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
